In [1]:
import pandas as pd
import numpy as np
import bamboolib as bam
import plotly.express as px
import requests
import json

In [17]:
colors = {'Accès fermé': 'rgb(38,40,63)',
          'Accès ouvert': 'rgb(255, 111, 76)',
          'Archive ouverte': 'rgb(25,144,91)',
          'Editeur': 'rgb(234,215,55)',
          'Editeur et archive ouverte':'rgb(145,174,79)'}
line_types = {'percent_acces_ouvert_20201130': 'dot',
          'percent_acces_ouvert_20210914': 'dash',
          'percent_acces_ouvert_20220504': 'solid'}
line_names = {'percent_acces_ouvert_20201130': '2020',
          'percent_acces_ouvert_20210914': '2021',
          'percent_acces_ouvert_20220504': '2022'}

In [4]:
def generate_pivot_table_year_oa_rate(df,obs_date):
    dp = pd.pivot_table(df, columns=['oa'],index=['year'], aggfunc='sum')
    dp = dp.reset_index()
    dp.columns = ["_".join([str(index) for index in multi_index]) for multi_index in dp.columns.ravel()]
    dp['year_'] = dp['year_'].astype('category')
    dp[f"percent_acces_ouvert_{obs_date}"] = round(dp['count_Accès ouvert'] / (dp['count_Accès ouvert'] + dp['count_Accès fermé']) * 100,2)
    return dp

In [5]:
dp1 = generate_pivot_table_year_oa_rate(df1,"20210914").drop(columns=['count_Accès fermé','count_Accès ouvert'])
dp2 = generate_pivot_table_year_oa_rate(df2,"20201130").drop(columns=['count_Accès fermé','count_Accès ouvert'])
dp = (generate_pivot_table_year_oa_rate(df,"20220504").merge(dp1, left_on='year_', right_on='year_',how='outer')).merge(dp2, left_on='year_', right_on='year_',how='outer')
dp

,year_,count_Accès fermé,count_Accès ouvert,percent_acces_ouvert_20220504,percent_acces_ouvert_20210914,percent_acces_ouvert_20201130
0,2016,833,1306,61.06,57.14,45.45
1,2017,778,1413,64.49,56.25,42.86
2,2018,792,1519,65.73,55.56,36.00
3,2019,725,1707,70.19,50.00,44.00
4,2020,752,1893,71.57,33.33,NaN
5,2021,842,1985,70.22,NaN,NaN


In [18]:
fig = px.line(dp.sort_values(by=['year_'], ascending=[True]).dropna(subset=['percent_acces_ouvert_20201130']), x='year_', y=['percent_acces_ouvert_20220504', 'percent_acces_ouvert_20201130'], line_dash_map=line_types, name=line_names, color_discrete_sequence=['rgb(255, 111, 76)'])
fig.update_traces(mode='lines+markers+text')
fig

TypeError: line() got an unexpected keyword argument 'name'

In [52]:
fig = px.line(dp.sort_values(by=['year_'], ascending=[True]), x='year_', y=['percent_acces_ouvert_20201130','percent_acces_ouvert_20210914','percent_acces_ouvert_20220504'])
fig.update_traces(connectgaps=False)

In [58]:
date = "20220504"
q = f"select count(doi) as count, is_oa_normalized as oa ,year as year from bso_publis_dedup_with_oa_20220504 where year <> {int(date[:4]) - 1} group by oa,year"
print(q)

select count(doi) as count, is_oa_normalized as oa ,year as year from bso_publis_dedup_with_oa_20220504 where year <> 2021 group by oa,year
